### Exploratory Data Analysis: 


### SetUp

Download datasets from Kaggle repository & import datasets into memory

In [3]:
import pandas as pd
import os
import kagglehub

path = kagglehub.dataset_download("nathanhg/uk-gam-datasets")
premises = '2024_premises-licence-register.csv'
premises_df = pd.read_csv(os.path.join(path,premises))


In [4]:
# Formatting columns
premises_df.columns = premises_df.columns.map(lambda col: col.replace(' ','_').lower())
premises_df.columns

Index(['account_number', 'account_name', 'premises_activity',
       'local_authority', 'address_line_1', 'address_line_2', 'city',
       'postcode'],
      dtype='object')

### Data Profiling

##### Overview

In [5]:
# from ydata_profiling import ProfileReport


# profile = ProfileReport(premises_df, title="Pandas Profiling Report", type_schema={
#         "account_number": "categorical",
#         "account_name": "categorical",
#         "premises_activity": "categorical",
#         "local_authority": "categorical",
#         "address_line_1": "categorical",
#          "address_line_2": "categorical",
#          "city": "categorical",
#         "postcode": "categorical",
#     })
# profile

❔ How many missing values are there? 
+ 💡Premises Activity has 10 missing values -we should rename these as 'Other'
+ 💡Local Authority has 8 missing values we could fill these based on WebScraped data.
+ 💡Address Line 2 has 20 missing values we should not replace these since they are Not Missing At Random (NMAR)

❔ Are there duplicate rows? 
+ 💡No  - the profiler identified similar records by with different cities and postcodes.  

❔ Which columns appear to be unique identifiers?
+ 💡Account Number

❔ Any functional dependencies?
+ 💡Yes, Account Number and Account Name have exactly 150 distinct values each and appear to have a 1-2-1 mapping. 


### Prototype Imputation Strategies

##### Missing Values

Premises Activity: Impute with 'Missing'

In [6]:
premises_df.fillna({'premises_activity':'Missing'},inplace=True) 
premises_df[premises_df['premises_activity'].isin(['Missing'])]

,account_number,account_name,premises_activity,local_authority,address_line_1,address_line_2,city,postcode
15,46777,SJRP Investments Ltd,Missing,NaN,25 Vicar Street,NaN,Kidderminster,DY10 1DA
114,42869,Amusetime Limited,Missing,Rushmoor Borough Council,Amusetime,25 Station Road,ALDERSHOT,GU11 1HT
148,31656,Chester Race Company Limited,Missing,Wrexham County Borough Council,"Bangorbet, The Racecourse","Overton Road, Bangor-on-Dee",WREXHAM,LL13 0DA
167,43440,Betextra Limited,Missing,NaN,Bet Extra,"Unit 7b, Crossgates Shopping,Centre",Leeds,LS15 8ET
663,1058,Done Brothers (Cash Betting) Limited,Missing,Liverpool City Council,Betfred,4 Jennifer Avenue,LIVERPOOL,L5 5AA
1691,45681,Frances Lucy D´Souza,Missing,London Borough of Haringey,Bigbux,72 West Green Road,LONDON,N15 5NS
1769,25086,Wray Brian Austrin,Missing,Great Yarmouth Borough Council,"Bubbles Amusements, 1 Warren Road",Hopton,GREAT YARMOUTH,NR31 9BN
1798,1969,Firstland Estates Limited,Missing,Huntingdonshire District Council,Careys Amusements,5-7 South Street,ST. NEOTS,PE19 2BW
1950,65112,Charlie Edwards Ltd,Missing,Swindon Borough Council,Charlies,120 Cricklade Road,Swindon,SN2 8AG
1952,31656,Chester Race Company Limited,Missing,Cheshire West and Chester Council,"Chesterbet, Chester Race Co Ltd",Watergate Square,CHESTER,CH1 2LY


✅ Premises Acivity Imputed

Local Authority: Impute with webscraped Local Authorities

In [7]:
mask = premises_df['local_authority'].isna() & premises_df['postcode'].notna()

postcodes_for_na_local_authorities = premises_df.loc[mask,'postcode'].to_list()

In [8]:
import json

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

responses = {}

for x, y in enumerate(postcodes_for_na_local_authorities):
    driver.get('https://www.gov.uk/find-local-council')
    post_code_entry = driver.find_element(By.NAME, 'postcode')
    post_code_entry.send_keys(y)
    find_button = driver.find_element(By.CSS_SELECTOR,'#local-locator-form > button')
    find_button.click()
    try:
        two_local_authorities = driver.find_element(By.CSS_SELECTOR,'#content > div > div.govuk-grid-column-two-thirds.article-container > h2')
        if two_local_authorities.text == 'Services in your area are provided by two local authorities':
            first_local_authority = driver.find_element(By.CSS_SELECTOR,'#content > div > div.govuk-grid-column-two-thirds.article-container > div > div.county-result.group.govuk-\!-margin-bottom-8 > h3')
            second_local_authority = driver.find_element(By.CSS_SELECTOR,'#content > div > div.govuk-grid-column-two-thirds.article-container > div > div.district-result.group > h3')
            responses[y] = f'{first_local_authority.text} / {second_local_authority.text}'
        else:
            try:
                local_council_answer = driver.find_element(By.CSS_SELECTOR,'#content > div > div.govuk-grid-column-two-thirds.article-container > div > div > p:nth-child(1) > strong')
                responses[y] = f'{local_council_answer.text}'
            except NoSuchElementException:
                responses[y] = 'No Local Council data Available'
    except NoSuchElementException:
        try:
            local_council_answer = driver.find_element(By.CSS_SELECTOR,'#content > div > div.govuk-grid-column-two-thirds.article-container > div > div > p:nth-child(1) > strong')
            responses[y]  = f'{local_council_answer.text}'
        except NoSuchElementException:
            responses[y] = 'No Local Council data available'

# Save to a JSON file
with open('missing_local_authorities.json', 'w') as file:
    json.dump(responses, file, indent=4)


In [10]:
import json

# Load previously scraped data
with open('config/missing_local_authorities.json', 'r') as file:
    responses = json.load(file)

# Validate data
print(responses)

# Update DataFrame
premises_df.loc[mask, 'local_authority'] = premises_df.loc[mask, 'postcode'].map(responses)

# Validate DataFrame
premises_df[premises_df.postcode.isin(responses.keys())]

FileNotFoundError: [Errno 2] No such file or directory: 'config/missing_local_authorities.json'

✅ Local Council Activity Imputed. (Consider index 2036 'TN34 1JY' has existing Local Authority values -- implement forward/backfill first?)

##### Drop Duplicate Rows

✅ We do not need these duplicates, we can drop these: 

In [11]:
premises_df.drop_duplicates(inplace=True)

✅ Or resolve them during the modelling stage

##### Erroneuos Values

In [86]:
# define function for fast similarity comparison between strings in a column

from rapidfuzz import fuzz
from itertools import combinations
def similar_strings(df,col,lowest_boundary=90):
    df = df.copy()
    matches_list = []
    df[col].sort_values()
    choices = df[col].dropna().unique().tolist()
    for i, j in combinations(range(len(choices)), 2):
        query_str = choices[i]
        matched_str = choices[j]
        
        # Calculate the similarity score between query_str and matched_str
        similarity_score = fuzz.WRatio(query_str, matched_str)
        
        # Only include matches with a similarity score between 90 and 100 (excluding exact matches)
        if similarity_score < 100 and similarity_score >= lowest_boundary:
            matches_list.append({
                f'{col}': query_str,
                f'matched_{col}': matched_str,
                'similarity_score': similarity_score
            })

    # Create a DataFrame from the collected matches
    matches_df = pd.DataFrame(matches_list)

    # Print the DataFrame
    return matches_df.sort_values('similarity_score',ascending=False)

❔ Are there any erroneous values for Account Name?

In [ ]:
col = 'account_name'
# General clean
premises_df[col] = premises_df[col].astype(str).apply(lambda x: x.strip())

# print similarities
print(similar_strings(premises_df,col))

# Validate
premises_df[premises_df.account_name.isin(['Grosvenor Casinos (GC) Limited','Grosvenor Casinos Limited'])]

                account_name            matched_account_name  similarity_score
0  Grosvenor Casinos Limited  Grosvenor Casinos (GC) Limited              95.0


,account_number,account_name,premises_activity,local_authority,address_line_1,address_line_2,city,postcode
40,614,Grosvenor Casinos Limited,Casino,Aberdeen City Council,Aberdeen Grosvenor Casino,5 Exchequer Row,ABERDEEN,AB11 5BW
1697,614,Grosvenor Casinos Limited,Casino,Birmingham City Council,Birmingham-Broad St Grosvenor Casino,"Fiveways Leisure Complex, Broad Street",BIRMINGHAM,B15 1AY
1698,34161,Grosvenor Casinos (GC) Limited,Casino,Birmingham City Council,Birmingham-Hill Street Grosvenor Casino,84 Hill Street,BIRMINGHAM,B5 4AH
1703,614,Grosvenor Casinos Limited,Casino,Blackpool Borough Council,Blackpool Grosvenor Casino,"The Sandcastle, Promenade",BLACKPOOL,FY4 1BB
1724,34161,Grosvenor Casinos (GC) Limited,Casino,Bournemouth Borough Council,Bournemouth Grosvenor Casino,48 Westover Road,BOURNEMOUTH,BH1 2BZ
1752,614,Grosvenor Casinos Limited,Casino,Brighton and Hove City Council,Brighton Grosvenor Casino,9 Grand Junction Road,BRIGHTON,BN1 1PP
1758,614,Grosvenor Casinos Limited,Casino,Bristol City Council,Bristol Grosvenor Casino,266 Anchor Road,BRISTOL,BS1 5TT
1796,614,Grosvenor Casinos Limited,Casino,City of Cardiff Council,Cardiff Grosvenor Casino,"The Red Dragon Centre, Atlantic Wharf, Hemingw...",CARDIFF,CF10 4JY


💡 No. There are two similar Account Names but with different Account Numbers

❔ Are there any erroneous values for Premises Activity?

In [ ]:
col = 'premises_activity'
# General clean
premises_df[col] = premises_df[col].astype(str).apply(lambda x: x.strip())

# print similarities
print(similar_strings(premises_df,col, lowest_boundary=85))

# Validate
premises_df[premises_df.premises_activity.isin(['Casino 2005'])]


  premises_activity matched_premises_activity  similarity_score
0            Casino               Casino 2005              90.0


,account_number,account_name,premises_activity,local_authority,address_line_1,address_line_2,city,postcode
127,26947,Aspers (Stratford City) Limited,Casino 2005,London Borough of Newham,ASPERS WESTFIELD STRATFORD CITY,312 THE LOFT WESTFIELD STRATFORD CITY MONTFICH...,LONDON,E20 1ET


In [89]:
# Replace
premises_df.premises_activity = premises_df.premises_activity.replace('Casino 2005','Casino')

# Validate
premises_df[premises_df.premises_activity.isin(['Casino 2005'])]

,account_number,account_name,premises_activity,local_authority,address_line_1,address_line_2,city,postcode


💡 Yes there was one category of Casino 2005 - likley refering to The Gambling Act 2005. This has been standardised to Casino due to the lack of frequency

❔ Are there any erroneous values for Local Authority?

In [90]:
col = 'local_authority'
# General clean
premises_df[col] = premises_df[col].astype(str).apply(lambda x: x.strip())

# print similarities
similar_strings(premises_df,col)

,local_authority,matched_local_authority,similarity_score
11,North East Lincolnshire Council,North Lincolnshire Council,95.000000
8,Renfrewshire Council,East Renfrewshire Council,95.000000
29,London Borough of Barnet,London Borough of Brent,93.617021
14,Woking Borough Council,Wokingham Borough Council,93.617021
3,Watford Borough Council,Dartford Borough Council,93.617021
19,South Kesteven District Council,North Kesteven District Council,93.548387
32,South Norfolk District Council,North Norfolk District Council,93.333333
21,East Dunbartonshire Council,West Dunbartonshire Council,92.592593
5,London Borough of Havering,London Borough of Haringey,92.307692
25,Warrington Borough Council,Darlington Borough Council,92.307692


💡 No the similarities are not an indication of erroneous values.

❔ Are there any erroneous values for Address Line 1?

In [ ]:
col = 'address_line_1'
# General clean
premises_df[col] = premises_df[col].astype(str).apply(lambda x: x.strip().lower())

# print similarities
similar_strings(premises_df,col)

,address_line_1,matched_address_line_1,similarity_score
14,caesars palace,caesar's palace,96.551724
12,boylesport,boylesports,95.238095
28,coastal amusements,coastal amusements limited,95.000000
19,carousel amusements,carousel amusements limited,95.000000
8,blue anchor leisure,blue anchor leisure limited,95.000000
7,betting shop operations,betting shop operations limited,95.000000
5,bet extra,betextra,94.117647
6,betszone,betzone,93.333333
26,club 2000 bingo,club 3000 bingo,93.333333
0,1 commercial street,45 commercial street,92.307692


In [92]:
# Replace
replacements = {
    "caesar's palace": "caesars palace",
    "boylesport": "boylesports",
    "coastal amusements limited": "coastal amusements",
    "carousel amusements limited" : "carousel amusements",
    "blue anchor leisure limited": "blue anchor leisure",
    "betting shop operations limited" : "betting shop operations", 
    "betextra" : "bet extra",
    "betszone" : "betzone",
    "a & s leisure group limited" : "a & s leisure group",
    "a & s leisure" : "a & s leisure group",
    "cashino gaming limited" : "cashino gaming"
}


premises_df.address_line_1 = premises_df.address_line_1.replace(replacements,regex=True)

# Validate
premises_df[premises_df.address_line_1.isin(replacements.keys())]

,account_number,account_name,premises_activity,local_authority,address_line_1,address_line_2,city,postcode


💡 Yes there were 11 address lines that required standardising - these will be checked again via Google API

❔ Are there any erroneous values for Address Line 2?

In [12]:
# General clean
col = 'address_line_2'
premises_df[col] = premises_df[col].astype(str).apply(lambda x: x.strip().lower())

# Save to csv for extensive comparison
similar_strings(premises_df,col,97)

NameError: name 'similar_strings' is not defined

In [ ]:
# Replace
replacements = {
    "coral island, unit 2 promenade": "coral island, unit 2, promenade",
    "98a -99 high street, gorleston": "98a-99 high street, gorleston",
    "36 -42 marine terrace	": "36-42 marine terrace"
}


premises_df.address_line_2 = premises_df.address_line_2.replace(replacements,regex=True)

# Validate
premises_df[premises_df.address_line_2.isin(replacements.keys())]

,account_number,account_name,premises_activity,local_authority,address_line_1,address_line_2,city,postcode


💡 Yes there were 3 address lines that required standardising - these will be checked again via Google API

❔ Are there any erroneous values for City?

In [49]:
# General clean
col = 'city'
premises_df[col] = premises_df[col].astype(str).apply(lambda x: x.strip().capitalize())

# Save to csv for extensive comparison
similar_strings(premises_df,col)

,city,matched_city,similarity_score
22,Middlesborough,Middlesbrough,96.296296
23,Burton-on-trent,Burton-upon-trent,93.750000
26,Blaydon-on-tyne,Blaydon-upon-tyne,93.750000
3,Nottingham,Cottingham,90.000000
4,Newcastle,Newcastle-upon-tyne,90.000000
5,Newcastle,Newcastle-under-lyme,90.000000
6,Bath,Bathgate,90.000000
7,Gillingham,Billingham,90.000000
0,Preston,Prestonpans,90.000000
1,Bradford,Bradford-on-avon,90.000000


In [50]:
# Replace
replacements = {
    "Middlesborough": "Middlesbrough",
    "Burton-on-trent": "Burton-upon-trent",
    "Blaydon-upon-tyne":"Blaydon-on-tyne",
    "Bury" : "Bury st. edmunds",
    "Lytham" : "Lytham st. annes",
    "Richmond":"Richmond-upon-thames"
}


premises_df.city = premises_df.city.replace(replacements,regex=True)

# Validate
premises_df[premises_df.city.isin(replacements.keys())]

,account_number,account_name,premises_activity,local_authority,address_line_1,address_line_2,city,postcode


💡 Yes there were 6 citys that required standardising, in many cases this is due to the old town spellings vs the new town spellings. For some UK history and reference point see below:
+ [Why is Middlesbrough sometimes misspelt?](https://www.thenorthernecho.co.uk/opinion/letters/17704426.letters-why-middlesbrough-sometimes-misspelt-middlesborough-/)
+ [The official name of the town is Burton upon Trent](https://en.wikipedia.org/wiki/Burton_upon_Trent#:~:text=According%20to%20the%20town's%20charter,for%20the%20town's%20railway%20station)
+ [If Blaydon-on-Tyne is good enough for the BBC, it's good enough for me](https://www.bbc.co.uk/news/topics/cwdnmkzrxlwt)
+ [Bury St Edmunds (/ˈbɛri sənt ˈɛdməndz/), commonly referred to locally as Bury](https://en.wikipedia.org/wiki/Bury_St_Edmunds)
+ [Lytham is the older settlement...reunited in 1922 under ... "Lytham St Annes"](https://en.wikipedia.org/wiki/Lytham_St_Annes#:~:text=Lytham%20is%20the%20older%20settlement,Ansdell%20being%20part%20of%20Lytham))
+ [The new borough was named 'Richmond upon Thames' rather than just Richmond](https://en.wikipedia.org/wiki/London_Borough_of_Richmond_upon_Thames#:~:text=The%20new%20borough%20was%20named,have%20occurred%20since%20its%20creation)

❔ Are there any erroneous values for postcode?

In [ ]:
# General clean
col = 'postcode'
premises_df[col] = premises_df[col].astype(str).apply(lambda x: x.strip())

# Save to csv for extensive comparison
similar_strings(premises_df,col)

,postcode,matched_postcode,similarity_score
0,WF18 1AL,WF8 1AL,93.333333
3,BH1 4AN,BH12 4AN,93.333333
1,N4 2HY,N4,90.000000
2,SN4 7AF,N4,90.000000
4,TN4 9TT,N4,90.000000
5,TN40 1DY,N4,90.000000
6,N4 2SB,N4,90.000000
7,WN4 9AN,N4,90.000000
8,DN40 2AB,N4,90.000000
9,NN4 8DW,N4,90.000000


💡 No, however some of these post codes look incomplete and will require varifying via Google API.

### Data Modelling

##### Relationships

❔ Why do some rows have the same Account Number?

In [52]:
premises_df[premises_df.account_number.isin([1034])]

,account_number,account_name,premises_activity,local_authority,address_line_1,address_line_2,city,postcode
0,1034,Power Leisure Bookmakers Limited,Betting Shop,London Borough of Tower Hamlets,1 commercial street,nan,London,E1 6BF
4,1034,Power Leisure Bookmakers Limited,Betting Shop,London Borough of Enfield,157 bramley road,oakwood,London,N14 4XA
9,1034,Power Leisure Bookmakers Limited,Betting Shop,Reading Borough Council,19 queen victoria street,nan,Reading,RG1 1SY
12,1034,Power Leisure Bookmakers Limited,Betting Shop,Sheffield City Council,212-214 london road,nan,Sheffield,S2 4LW
16,1034,Power Leisure Bookmakers Limited,Betting Shop,London Borough of Hackney,260 seven sisters road,finsbury park,London,N4 2HY
18,1034,Power Leisure Bookmakers Limited,Betting Shop,London Borough of Harrow,284 – 286 northolt road,south harrow,Middlesex,HA2 8EB
27,1034,Power Leisure Bookmakers Limited,Betting Shop,London Borough of Enfield,62 fore street,edmonton,London,N18 2SW
31,1034,Power Leisure Bookmakers Limited,Betting Shop,London Borough of Havering,91 south street,romford,Essex,RM1 1NX


❔ Do some Addresses have different Account Numbers?

In [53]:
premises_df[premises_df.index.isin([1992,1993])]

,account_number,account_name,premises_activity,local_authority,address_line_1,address_line_2,city,postcode
1992,52732,City Gaming Limited,Adult Gaming Centre,nan,city slots,88 high street bromley,London,BR1 1HF
1993,1875,Family Leisure Holdings Limited,Adult Gaming Centre,nan,city slots,88 high street bromley,London,BR1 1HF


💡💡 Account Numbers have a Many-to-Many relationship with Premises Addresses. 

❔ Do some Accounts have different Premises Activities (licences)?

In [54]:
premises_df[premises_df.account_number.isin([2795])]

,account_number,account_name,premises_activity,local_authority,address_line_1,address_line_2,city,postcode
2063,2795,Isabelle Marianne Belton,Family Entertainment Centre,East Lindsey District Council,coral beach club,"skegness road, ingoldmells",Skegness,PE25 1JW
2064,2795,Isabelle Marianne Belton,Adult Gaming Centre,East Lindsey District Council,coral beach club,"skegness road, ingoldmells",Skegness,PE25 1JW
2065,2795,Isabelle Marianne Belton,Bingo,East Lindsey District Council,coral beach club,"skegness road, ingoldmells",Skegness,PE25 1JW


💡 Account Numbers have a Many-to-Many relationship with Premises Activities

##### SQL Implementation

In [55]:
# Defining a uniqueness test

def test_distinct_id(df,id):
    """
    Function for testing uniqueness of a field within a dataframe
    params:
    df(pd.Dataframe): The dataframe being tested
    id: The field being tested for uniqueness
    """
    value_counts = df[id].value_counts().reset_index()
    value_counts.columns = [id, 'count']
    duplicates= value_counts[value_counts['count']>1]
    if duplicates.empty:
        print(f'{id} is distinct')
        return None
    else:
        print('Duplicate IDs found:')
        print(duplicates)
        return None

EDR identifies key entities and explains the relationships between them 

In [56]:
import duckdb 

# Create dim_accounts table  
da_query =  """ 
SELECT 
    DISTINCT 
    account_number as accountid,	
    account_name
FROM premises_df
"""
dim_account = duckdb.query(da_query).to_df()

# Unique ID test

test_distinct_id(dim_account,'accountid')


ModuleNotFoundError: No module named 'duckdb'

In [181]:
dp_query = """
SELECT 
    DISTINCT
    md5(concat(address_line_1,address_line_2,city,postcode)) AS premisesid,
    address_line_1,
    address_line_2,
    city,
    postcode,
    local_authority
FROM premises_df
"""
dim_premises = duckdb.query(dp_query).to_df()

test_distinct_id(dim_premises,'premisesid')

premisesid is distinct


In [182]:
fl_query = """
WITH fact_premises_licences AS(
SELECT 
    DISTINCT
    md5(concat(address_line_1,address_line_2,city,postcode)) AS premisesid,
    premises_activity as activity,
    account_number AS accountid
FROM premises_df
)

SELECT
    md5(concat(premisesid, accountid,activity)) AS premises_licenceid,
    premisesid,
    accountid,
    activity
FROM fact_premises_licences
"""
fact_premises_licences = duckdb.query(fl_query).to_df()

test_distinct_id(fact_premises_licences,'premises_licenceid')

premises_licenceid is distinct


In [183]:
query = """
SELECT
    premisesid,
    string_agg(DISTINCT activity, '/') as licenced_activities,
    string_agg(DISTINCT accountid, '/') as account_ids
FROM fact_premises_licences
GROUP BY premisesid
"""
agg_test = duckdb.query(query).to_df()

test_distinct_id(agg_test,'premisesid')

premisesid is distinct


In [184]:
print(test_distinct_id(fact_premises_licences,'premisesid'))
agg_test[agg_test['premisesid']=='42b2ea841a2550fee15e0ca3b637c02b']

Duplicate IDs found:
                          premisesid  count
0   fc259698f2b50fb8428defb0295946a1      3
1   42b2ea841a2550fee15e0ca3b637c02b      3
2   78e1034250f26812f374e21c88531098      2
3   afc68466c82e1feb6c9998bd6039f805      2
4   3a372cd96dafa86594f48d6a7257acff      2
5   32d0c2f31003fb7e72b8bf0af349cc6e      2
6   2438bf973ec5fe777f3db7f700a80445      2
7   e5cc55706d1354e24e6232b9a6eb10d6      2
8   a904167e09aadddc864fa543a0cda2e7      2
9   c2a18ff0044708a6c3178d1a5ec838d3      2
10  0cc030ebaf287f537f20db89f7da7cfe      2
11  b046f48ccefc048fa72512c8b1601e9f      2
12  e46cc4e58bc84241ffefeb104109def6      2
13  d8299ee1e7ff64ff86d3af6b100da831      2
14  99951f5d4e02fa4355ab29ca2006a4af      2
None


,premisesid,licenced_activities,account_ids
31,42b2ea841a2550fee15e0ca3b637c02b,Family Entertainment Centre/Adult Gaming Centr...,2795
